# Table of Contents

- [Goal](#Goal)
- [Wrangle](#Wrangle)
- [EDA](#Explore-Data-Analysis-(EDA))
  - [Visualizations](#Visualization)
    - [En Masse](#En-Masse)
    - [Insightful](#Insightful-Visualizations)
  - [Hypthosis Testing](#Hypthosis-Testing)
- [Choosen Features](#Choosen-Features)
- [Modeling](#Modeling)
- [Results and Conclusions](#Results-&-Conclusion)
  - [Next Steps](#Next-Steps)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from math import sqrt


import scipy.stats as stats

from wranglev2 import wrangle_credit

import warnings
warnings.filterwarnings("ignore")
print('Initialized')

Initialized


# Goal

The goal of this project is to...

# Wrangle

All the prep/wrangle work is handled by the wrangle_credit function imported above. For an indpeth explination of how the wrangle_credit function works please see the [How-To Notebook](https://github.com/credit-risk-management-predictor/credit-risk-management-predictor/blob/main/how_to.ipynb).

In [2]:
help(wrangle_credit)

Help on function wrangle_credit in module wranglev2:

wrangle_credit()
    This function does the following:
    * uses get_reports_data function to process credit_reports.csv data into the expanded DF 
    * uses get_application_data functionto process the application_record.csv into an apps DF
    * uses encode_dummies function to create dummy variables from the categorical variables of the apps DF
    * merges the apps and expanded DFs into a final_df on 'id'
    * uses split_stratify_data on the final_df to create the train, validate, test data sets
    * return train, validate, test data sets as DFs
    Note - The create_scaled_x_y function will be used after EDA to avoid confusion.
    Note 2 - Remember to drop columns not used as features such as age, gender, etc



# EDA

## Visualizations

### Insightful Visualizations

## Hypthosis Testing

# Choosen Features

# Modeling

# Results and Conclusions

## Next Steps

# Appendix